# Optimización de Rastrigin por Enjambre de Partículas
## Definición de la función

In [185]:
import numpy as np

K_DIMENSION = 3
K_A = 10

def rastrigin(*args: np.ndarray) -> np.ndarray:
    valores = np.array([args[i] for i in range(0, len(args))])
    valores = valores.transpose() if valores.shape[0] > 1 else valores
    sumando = np.sum(np.power(valores, 2) - K_A * np.cos(2*np.pi * valores), axis=-1)

    return K_A * 3 + sumando

# Definición de Partícula

In [186]:
K_LIMITE_SUPERIOR = 5
K_LIMITE_INFERIOR = -5

class Particle:
    def __init__(self):
        dominio = K_LIMITE_SUPERIOR - K_LIMITE_INFERIOR

        self._dimension = K_DIMENSION
        self.coordenadas = K_LIMITE_INFERIOR + dominio*np.random.random(size=self._dimension)
        self.memoria = np.copy(self.coordenadas)
        self.velocidad = -dominio/2 + dominio*np.random.random(size=self._dimension)

    def __str__(self):
        return str(self.coordenadas)

    def to_evaluate(self, use_coors=True):
        if use_coors:
            return [np.array(self.coordenadas[dim]) for dim in range(self._dimension)]
        else:
            return [np.array(self.memoria[dim]) for dim in range(self._dimension)]

    def get(self, index: int):
        return self.coordenadas[index]


# Definición de Enjambre

In [187]:
from typing import Callable

K_COEFICIENTE_PESO = 0.8
K_COEFICIENTE_MEMORIA = 0.4
K_COEFICIENTE_LIDER = 0.8

class Swarm:
    def __init__(self, num_particulas: int, function: Callable[..., np.ndarray]):
        self._num_particulas = num_particulas
        self._poblacion = [Particle() for _ in range(num_particulas)]
        self._coef_peso = 0.5
        self._num_generacion = 0
        self._function = function
        self._init_best()

    def _init_best(self):
        self._mejor : Particle
        salida = self._evaluar()
        index = salida.argmin()
        self._mejor = self._poblacion[index]

    def _evaluar(self) -> np.ndarray:
        coordenadas = self._get_coordenadas()
        return self._function(*coordenadas)

    def _get_coordenadas(self) -> list[np.ndarray]:
        return [np.array([particula.get(dim) for particula in self._poblacion]) for dim in range(K_DIMENSION)]

    def _actualizar_particula(self, particula: Particle):
        actual = self._function(*particula.to_evaluate(use_coors=True))
        ultimo_mejor = self._function(particula.to_evaluate(use_coors=False))

        if actual < ultimo_mejor:
            particula.memoria = np.copy(particula.coordenadas)
            mejor_del_enjambre = self._function(self._mejor.to_evaluate(use_coors=False))

            if ultimo_mejor < mejor_del_enjambre:
                self._mejor = particula

    def iterar(self):
        self._num_generacion += 1
        self._print_generation()

        for particula_index in range(self._num_particulas):
            particula: Particle = self._poblacion[particula_index]

            for dim in range(K_DIMENSION):
                coef_inercia = K_COEFICIENTE_PESO * particula.velocidad[dim]
                coef_memoria = K_COEFICIENTE_MEMORIA * np.random.random() * (particula.memoria[dim] - particula.coordenadas[dim])
                coef_lider = K_COEFICIENTE_LIDER * np.random.random() * (self._mejor.memoria[dim] - particula.coordenadas[dim])

                particula.velocidad[dim] = coef_inercia + coef_memoria + coef_lider
                particula.coordenadas[dim] = particula.coordenadas[dim] + particula.velocidad[dim]

            self._actualizar_particula(particula)

    def _print_generation(self):
        valor = self._function(self._mejor.to_evaluate(use_coors=False))

        print(f"Generation {self._num_generacion}")
        print("Best so far:")
        print(f"Coordenates: {self._mejor}")
        print(f"Function value: {valor}")

# Optimización
## Inicialización

In [188]:
K_NUM_PARTICULAS = 20
enjambre = Swarm(K_NUM_PARTICULAS, rastrigin)

## Iteración

In [189]:
enjambre.iterar()

Generation 1
Best so far:
Coordenates: [ 2.94041359 -1.71521161 -1.08886296]
Function value: [27.15341616]
Generation 2
Best so far:
Coordenates: [ 3.36499983 -4.93478444 -3.62500203]
Function value: [27.15341616]
Generation 3
Best so far:
Coordenates: [ 3.52944246 -5.59629412 -4.23161405]
Function value: [27.15341616]
Generation 4
Best so far:
Coordenates: [ 0.0987413   0.19015431 -0.10437211]
Function value: [10.32250169]
Generation 5
Best so far:
Coordenates: [ 1.64786176  1.11384829 -0.71210126]
Function value: [10.32250169]
Generation 6
Best so far:
Coordenates: [ 1.93378934  1.10769091 -0.58126003]
Function value: [10.32250169]
Generation 7
Best so far:
Coordenates: [ 1.44528223  0.69896345 -0.21374501]
Function value: [10.32250169]
Generation 8
Best so far:
Coordenates: [ 0.11477809 -0.04106878  0.17144568]
Function value: [8.12727718]
Generation 9
Best so far:
Coordenates: [-0.94962523 -0.63309456  0.47959823]
Function value: [8.12727718]
Generation 10
Best so far:
Coordenates: